In [21]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [22]:
load_dotenv(override=True)

True

In [23]:
model = "gpt-4o-mini"

In [24]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [52]:
sales_agent1 = Agent(
    name="Professional Sales Agent",
    instructions=instructions1,
    model=model,
    output_type=str
)

sales_agent2 = Agent(
    name="Engaging Sales Agent",
    instructions=instructions2,
    model=model,
    output_type=str
)

sales_agent3 = Agent(
    name="Busy Sales Agent",
    instructions=instructions3,
    model=model,
    output_type=str
)

In [26]:
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email.")

async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Simplify Your SOC 2 Compliance Process

Hi [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I’m with ComplAI—a leading provider of software solutions designed to streamline SOC 2 compliance and audit preparation.

With the increasing complexity of regulatory requirements, we understand that maintaining compliance can be a daunting task. Our AI-powered tool simplifies this process, providing real-time insights and automating documentation, which not only saves time but also enhances accuracy.

We’ve helped companies like [Client A] and [Client B] reduce their audit prep time by up to 50%, allowing them to focus on what they do best—serving their clients.

I would love the opportunity to discuss how ComplAI can support your compliance efforts and help you stay ahead of regulatory challenges. Are you available for a brief call next week?

Thank you for your time, and I look forward to the possibility of working together.

Best regards,

[Your N

In [27]:
message = "Write a cold sales email."

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")

Subject: Simplify Your SOC2 Compliance Journey with ComplAI

Hi [Recipient's Name],

I hope this message finds you well.

Navigating the complexities of SOC2 compliance can be daunting. At ComplAI, we understand the challenges organizations face in preparing for audits while maintaining security and operational effectiveness. 

Our AI-powered SaaS solution is designed to streamline the compliance process, helping you identify gaps, automate documentation, and ensure you meet regulatory requirements without the burden of manual oversight. With features tailored to simplify compliance management, you'll gain the confidence needed to pass audits with ease.

I’d love to schedule a brief call to discuss how ComplAI can support your compliance efforts and enhance your organization’s overall security posture. Are you available for a quick chat this week?

Thank you for your time, and I look forward to the possibility of working together.

Best regards,

[Your Name]  
[Your Position]  
ComplAI

In [28]:
sales_picker = Agent(
    name="sales picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model=model
)

In [29]:
message = "Write a cold sales email."

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best cold sales email:\n{best.final_output}")

Best cold sales email:
Subject: Let's Make Compliance Your New Best Friend! 🤝

Hey [First Name],

Hope your day is as smooth as a freshly brewed cup of coffee! ☕️

I know you’re busy juggling reports and deadlines, but let’s be real—SOC2 compliance is that frenemy who keeps popping up when you least expect it. It’s time to change the narrative!

At ComplAI, we’ve developed a SaaS tool that transforms compliance from a stressful chore to a breezy walk in the park (with free Wi-Fi, of course). Our AI-powered solution not only simplifies your audit prep but also ensures you stay compliant while enjoying life… or at least enjoying that second donut! 🍩

Here are a couple of things we can help you with:

1. **Automated Compliance Checks**: Because who has time for manual audits?
2. **Real-Time Reporting**: Know exactly where you stand… and brag about it at the next office happy hour!

Curious to learn more? Let’s schedule a quick 15-minute chat (or just swap some bad puns—your choice!). 🎤

L

In [59]:
@function_tool
def send_email(body: str):
    print("send_email called with body:", body)
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("kaya.atsy@gmail.com")
    to_email = To("kaya.atsy@gmail.com")
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    print("Built mail object:", mail)
    response = sg.client.mail.send.post(request_body=mail)
    print("SendGrid response:", response.status_code, response.body)
    return "Email successfully sent"


In [60]:
send_email

FunctionTool(name='send_email', description='', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000020EE43891C0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

In [61]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email.")
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000020EE438AD40>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

In [62]:
description = "Write a cold sales email."

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000020EE438B2E0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent2', description='Write a cold sales email.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000020EE438B380>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent3', des

In [63]:
instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""

In [64]:
sales_manager = Agent(name="Sales manager", instructions=instructions, tools=tools, model=model)

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales Manager"):
    result = await Runner.run(sales_manager, message)

send_email called with body: Subject: Enhance Your SOC2 Compliance with ComplAI

Dear [CEO's Name],

I hope this email finds you well.

As organizations face increasing demands for transparency and security, maintaining SOC2 compliance is essential. At ComplAI, we offer a powerful SaaS solution that simplifies and automates the compliance process, leveraging AI to ensure you meet audit requirements with ease.

Our platform not only streamlines your compliance management but also enhances your operational efficiency, ultimately saving you both time and resources. Clients have reported significant improvements in their audit readiness and overall compliance posture after implementing our tool.

I would appreciate the opportunity to discuss how ComplAI can help your organization navigate the complexities of SOC2 compliance. Would you be available for a brief conversation next week?

Best regards,

[Your Name]  
[Your Position]  
ComplAI  
[Your Phone Number]  
[Your Email]  
Built mail ob

In [65]:
#Handoffs

In [93]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design.\
When converting to HTML, pay extra attention to any placeholder text inside the brackets.\
If such placeholder brackets exist in the body passed to you, simply change them to a more general word according to\
the info inside the brackets and the overall tone of the email body.\
For example, if you see [CEO's Name] in a body, you must simply change it to CEO without brackets."

In [94]:
subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model=model)
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model=model)
html_converter_tool = html_converter.as_tool(tool_name="html_converter", tool_description="Convert a text email body to an HTML email body")

In [95]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("kaya.atsy@gmail.com")
    to_email = To("kaya.atsy@gmail.com")
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return "Successfully sent email"

In [96]:
tools = [subject_tool, html_converter_tool, send_html_email]

In [97]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."

emailer_agent = Agent(name="Email Manager", instructions=instructions, model=model, tools=tools, handoff_description="Convert an email to HTML and send it")

In [98]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]

In [ ]:
sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools a second time to draft new email bodies if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""

In [100]:
sales_manager = Agent(name="Sales Manager", instructions=sales_manager_instructions, model=model, tools=tools, handoffs=handoffs)

message = "Send out a cold sales email addressed to 'Dear CEO' from Alice. Make sure email bodies do not contain placeholder text/brackets."

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)